<a href="https://colab.research.google.com/github/manki8908/firstcompany/blob/master/15_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import folium
import matplotlib.pyplot as plt
import matplotlib as mpl

In [41]:
df = pd.read_csv('/content/자치구별 공원 현황.csv')
df.head()

,지역,공원면적,공원수,인구,면적,면적비율,인당면적
0,강남구,3626398,7,544055,39500000.0,9.180754,6.665499
1,강동구,1526390,7,463998,24590000.0,6.207361,3.289648
2,강북구,3714902,4,311569,23600000.0,15.741110,11.923208
3,강서구,2194154,9,585901,41440000.0,5.294773,3.744923
4,관악구,10431453,2,509803,29570000.0,35.277149,20.461733


# 1. 자치구별 면적대비/인구대비 공원 면적

## 맵 로드

In [42]:
# 동별 geo 데이터
geo_path = '/content/seoul-dong.geojson'
try: 
    df_geo = json.load(open(geo_path, encoding='utf-8'))
except:
    df_geo = json.load(open(geo_path, econding='utf-8-sig'))   # return dict

df_geo.keys()
#df_geo
df_geo['features'][0]

{'type': 'Feature',
 'properties': {'시': '서울특별시', '구': '종로구', '동': '사직동', '행정동코드': 11110530},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[126.97688884274817, 37.575650779448786],
     [126.9770344988775, 37.569194530054546],
     [126.97597472821249, 37.569336299425764],
     [126.97537470991254, 37.56931556702156],
     [126.97433193562325, 37.56926180051753],
     [126.96904837001854, 37.56819441770833],
     [126.96854493603384, 37.56842767961276],
     [126.9666499598212, 37.569491655206576],
     [126.96628175024485, 37.5697007347987],
     [126.9660973270804, 37.5698565097237],
     [126.96572852922577, 37.570183936115114],
     [126.96592699822128, 37.5703188056862],
     [126.96601094018429, 37.57154839557748],
     [126.96365922052196, 37.575174660660366],
     [126.9630860043451, 37.57648592001554],
     [126.96284099051198, 37.57666158609274],
     [126.96281041047263, 37.57944880965677],
     [126.96742431584332, 37.57960153712449],
     [126.96742176302651, 

In [43]:
# 구별 geo 데이터
geo_gu_path = '/content/seoul-gu-simple.json'
try: 
    df_geo_gu = json.load(open(geo_gu_path, encoding='utf-8'))
except:
    df_geo_gu = json.load(open(geo_gu_path, econding='utf-8-sig'))   # return dict

df_geo_gu.keys()
df_geo_gu['features'][0]

{'type': 'Feature',
 'id': '강동구',
 'properties': {'code': '11250',
  'name': '강동구',
  'name_eng': 'Gangdong-gu',
  'base_year': '2013'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[127.11519584981606, 37.557533180704915],
    [127.16683184366129, 37.57672487388627],
    [127.18408792330152, 37.55814280369575],
    [127.16530984307447, 37.54221851258693],
    [127.14672806823502, 37.51415680680291],
    [127.12123165719615, 37.52528270089],
    [127.1116764203608, 37.540669955324965],
    [127.11519584981606, 37.557533180704915]]]}}

In [44]:
# 데이터 처리
new_df1 = df[['지역','면적비율']]

# 데이터 처리
new_df2 = df[['지역','인당면적']]

print(df.info())


# 위경도 데이터
ll = pd.read_csv('서울공원요약.csv', encoding='cp949')
ll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      25 non-null     object 
 1   공원면적    25 non-null     int64  
 2   공원수     25 non-null     int64  
 3   인구      25 non-null     int64  
 4   면적      25 non-null     float64
 5   면적비율    25 non-null     float64
 6   인당면적    25 non-null     float64
dtypes: float64(3), int64(3), object(1)
memory usage: 1.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   공원명     130 non-null    object 
 1   지역      130 non-null    object 
 2   경도      130 non-null    float64
 3   위도      130 non-null    float64
 4   면적      130 non-null    int64  
 5   스케일     130 non-null    object 
 6   크기      130 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 7.2+ KB

## 단계구분도

### 단계구분도 - 자치구별 면적비율 

In [45]:
# 맵
map = folium.Map(location=[37.581,126.986], zoom_start=11, tiles='Stamen Toner')
#map = folium.Map(location=[37.581,126.986], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map)

# 단계구분도
folium.Choropleth(geo_data = df_geo_gu,                     # 지도 데이터 파일
                  data = new_df1,                         # 시각화 데이터 파일
                  columns = ('지역', '면적비율'),              # (지도 데이터와 맵핑할 값, 시각화 하고자하는 변수)
                  key_on='feature.properties.name',        # 데이터파일과 매칭할 값
                  fill_color = 'BuPu', 
                  legend_name = '면적비율',
                  ).add_to(map)
title = '<h3 align="center" style="font-size:20px"> 면적비율</h3>'
map.get_root().html.add_child(folium.Element(title))
map

### 단계구분도 - 자치구별 인구비율 

In [46]:
# 맵
map = folium.Map(location=[37.581,126.986], zoom_start=11, tiles='Stamen Toner')
#map = folium.Map(location=[37.581,126.986], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map)

# 단계구분도
folium.Choropleth(geo_data = df_geo_gu,                     # 지도 데이터 파일
                  data = new_df2,                         # 시각화 데이터 파일
                  columns = ('지역', '인당면적'),              # (지도 데이터와 맵핑할 값, 시각화 하고자하는 변수)
                  key_on='feature.properties.name',        # 데이터파일과 매칭할 값
                  fill_color = 'BuPu', 
                  legend_name = '인당면적',
                  ).add_to(map)
title = '<h3 align="center" style="font-size:20px"> 인당면적</h3>'
map.get_root().html.add_child(folium.Element(title))
map

### 공원위치 추가표시

In [47]:
# 공원크기 
for i in ll.index:
  folium.Circle(radius=int(ll.크기[i]*50),
                location=[ll.위도[i], ll.경도[i]],
                popup=folium.Popup(ll.공원명[i], max_width=200),
                tooltip = f"{ll.공원명[i]}({ll.면적[i]: ,d})㎡",
                color='red',
                fill = True,
                fill_color='#3186cc').add_to(map)
map